In [ ]:
import numpy as np
import math
import glob

In [ ]:
temperature = 298.15 # K
boltzmann = 0.0019872041 # kcal/mol K
beta = 1.0/(boltzmann*temperature)

In [ ]:
# edit if needed

k_boresch = 100 # kcal/mol rad**2
k_rmsd = 100 # kcal/mol A**2
sep_cv_max = 6 # nm

In [ ]:
standard_volume = 1660 # angstroms^3
standard_volume_nm = standard_volume*0.001 # nm^3
radius_sphere = (3*standard_volume_nm/(4*np.pi))**(1.0/3.0) # radius of sphere whose volume is equal to the standard volume in nm

In [ ]:
def readPMF(file): # read PMF file CV range and Free Energy data
    
    data = np.loadtxt(file)
    x = data[:,0]
    y = data[:,1]

    return np.array((x, y))

In [ ]:
def geometric_restraint(pmf, k_restraint, rmsd=False, unbound=False):
        
        width = pmf[0][1] - pmf[0][0]

        if rmsd:
            restraint_center = 0 # for the RMSD restraint, the minimum is 0
        else:
            restraint_center = pmf[0][np.argmin(pmf[1])] # the minimum of the PMF from the PMF file
            

        numerator = 0
        denominator = 0
        for x, y in zip(pmf[0], pmf[1]):
            numerator += width*math.exp(-beta*y)
            denominator += width*math.exp((-beta)*(y+0.5*k_restraint*((x-restraint_center)**2)))
        
        contribution = math.log(numerator/denominator)/beta

        if unbound:
            return contribution
        else:
            return -contribution

In [ ]:
def separation_pmf(pmf, r_star):
    

    w_r_star = pmf[1][0]
    for x, y in zip(pmf[0], pmf[1]):
        if x >= r_star:
            w_r_star = y
            break
        
    width = pmf[0][1] - pmf[0][0]
    I = 0
    for x, y in zip(pmf[0], pmf[1]):
        I += width*math.exp(-beta*(y-w_r_star))
        if x >= r_star:
            break

    return -1/(beta)*math.log(3*I/radius_sphere)

In [ ]:
def boresch_correction(r_star, theta_a_min, theta_b_min, k_boresch):
    corr = (r_star**2)*math.sin(theta_a_min)*math.sin(theta_b_min)*(2*np.pi/beta)**2.5/(8*(np.pi**2)*(4*np.pi*radius_sphere**2)*(k_boresch)**2.5)

    return -1/(beta)*math.log(corr)

In [ ]:
def dg_contributions_restraints(CV_dir, k_restraint, rmsd=False, unbound=False):
    dg_list = []

    replicate_dirs = 'replicate-*/cv_fe_norm_kcal.txt'
    
    for rep_dir in glob.glob(CV_dir+replicate_dirs):
        pmf = readPMF(rep_dir)
        dg = geometric_restraint(pmf, k_restraint, rmsd, unbound)
        dg_list.append(dg)
    
    return(dg_list)

In [ ]:
def dg_contributions_separation(CV_dir, r_star):

    dg_list = []

    replicate_dirs = 'replicate-*/cv_fe_norm_kcal.txt'

    for rep_dir in glob.glob(CV_dir+replicate_dirs):
        pmf = readPMF(rep_dir)
        dg = separation_pmf(pmf, r_star)
        dg_list.append(dg)
    
    return(dg_list)

RMSD - Bound State

In [ ]:
rmsd_bound_dir = 'rmsd_bound/'
found_rmsd_bound = glob.glob(f'**/{rmsd_bound_dir}/', recursive=True)

if found_rmsd_bound:
    rmsd_bound = True
else:
    rmsd_bound = False

In [ ]:
rmsd_bound

In [ ]:
if rmsd_bound:
    dg_rmsd_bound_list = dg_contributions_restraints(rmsd_bound_dir, k_rmsd, rmsd=True)

    dg_rmsd_bound = (round(np.mean(dg_rmsd_bound_list), 2))

    print(str(round(np.mean(dg_rmsd_bound_list), 2))+' +- '+str(round(np.std(dg_rmsd_bound_list)/np.sqrt(len(dg_rmsd_bound_list)), 2)))

Boresch Theta A

In [ ]:
dg_theta_a_list = dg_contributions_restraints('1_boresch_theta_a/', k_boresch)
dg_theta_a_list

In [ ]:
dg_theta_a = (round(np.mean(dg_theta_a_list), 2))

print(str(round(np.mean(dg_theta_a_list), 2))+' +- '+str(round(np.std(dg_theta_a_list)/np.sqrt(len(dg_theta_a_list)), 2)))

Boresch Theta B

In [ ]:
dg_theta_b_list = dg_contributions_restraints('2_boresch_theta_b/', k_boresch)
dg_theta_b_list

In [ ]:
dg_theta_b = (round(np.mean(dg_theta_b_list), 2))

print(str(round(np.mean(dg_theta_b_list), 2))+' +- '+str(round(np.std(dg_theta_b_list)/np.sqrt(len(dg_theta_b_list)), 2)))

Boresch Phi A

In [ ]:
dg_phi_a_list = dg_contributions_restraints('3_boresch_phi_a/', k_boresch)
dg_phi_a_list

In [ ]:
dg_phi_a = (round(np.mean(dg_phi_a_list), 2))

print(str(round(np.mean(dg_phi_a_list), 2))+' +- '+str(round(np.std(dg_phi_a_list)/np.sqrt(len(dg_phi_a_list)), 2)))

Boresch Phi B

In [ ]:
dg_phi_b_list = dg_contributions_restraints('4_boresch_phi_b/', k_boresch)
dg_phi_b_list

In [ ]:
dg_phi_b = (round(np.mean(dg_phi_b_list), 2))

print(str(round(np.mean(dg_phi_b_list), 2))+' +- '+str(round(np.std(dg_phi_b_list)/np.sqrt(len(dg_phi_b_list)), 2)))

Boresch Phi C

In [ ]:
dg_phi_c_list = dg_contributions_restraints('5_boresch_phi_c/', k_boresch)
dg_phi_c_list

In [ ]:
dg_phi_c = (round(np.mean(dg_phi_c_list), 2))

print(str(round(np.mean(dg_phi_c_list), 2))+' +- '+str(round(np.std(dg_phi_c_list)/np.sqrt(len(dg_phi_c_list)), 2)))

Separation

In [ ]:
dg_sep_list = dg_contributions_separation('6_separation/', sep_cv_max - 0.5)
dg_sep_list

In [ ]:
dg_sep = (round(np.mean(dg_sep_list), 2))

print(str(round(np.mean(dg_sep_list), 2))+' +- '+str(round(np.std(dg_sep_list)/np.sqrt(len(dg_sep_list)), 2)))

RMSD - Unbound State

In [ ]:
rmsd_unbound_dir = 'rmsd_unbound/'
found_rmsd_unbound = glob.glob(f'**/{rmsd_unbound_dir}/', recursive=True)

if found_rmsd_unbound:
    rmsd_unbound = True
else:
    rmsd_unbound = False

In [ ]:
rmsd_unbound

In [ ]:
if rmsd_unbound:
    dg_rmsd_unbound_list = dg_contributions_restraints(rmsd_unbound_dir, k_rmsd, rmsd=True, unbound=True)
    dg_rmsd_unbound_list

    dg_rmsd_unbound = (round(np.mean(dg_rmsd_unbound_list), 2))

    print(str(round(np.mean(dg_rmsd_unbound_list), 2))+' +- '+str(round(np.std(dg_rmsd_unbound_list)/np.sqrt(len(dg_rmsd_unbound_list)), 2)))

Restraint Correction

In [ ]:
# equilibrium values, modify if needed

theta_a_min = 45.0*np.pi/180.0
theta_b_min = 104.0*np.pi/180.0

In [ ]:
restraints_corr = boresch_correction(sep_cv_max-0.5, theta_a_min, theta_b_min, k_boresch)
restraints_corr

Free Energy of Binding

In [ ]:
if rmsd_bound == False and rmsd_unbound == False:
    dg_bind = np.mean(dg_theta_a) + np.mean(dg_theta_b) + np.mean(dg_phi_a) + np.mean(dg_phi_b) + np.mean(dg_phi_c) + np.mean(dg_sep) + restraints_corr

else:
    dg_bind = np.mean(dg_rmsd_bound) + np.mean(dg_theta_a) + np.mean(dg_theta_b) + np.mean(dg_phi_a) + np.mean(dg_phi_b) + np.mean(dg_phi_c) + np.mean(dg_sep) + np.mean(dg_rmsd_unbound) + restraints_corr

In [ ]:
dg_bind